# Hospitality Opportunities in HS2-Era Birmingham

5 March 2020 / W Carter

This notebook is part of a final submission for the IBM Data Science Professional certificate course.  The business problem is fictitious, and intended only to demonstrate competence with data science methods.  The notebook was produced on IBM Watson Studio, before being 'pushed' to this online repository.

## Introduction & Business Problem

A new rail infrastructure project has recently been confirmed in the UK, the 'HS2'.  This ambitious transport system will connect the capital, London, to the second-most populous city in the UK, Birmingham (and the greater conurbation in the West Midlands), and later to northern England and Scotland.  Rail transport times between London and Birmingham will be reduced from 1hr 24 mins to just 49 mins, over a similar time period that increasing environmental pressure is building to reduce on air travel.

Birmingham City plans to capitalise on the transport upgrade for massive economic regeneration, and are mobilising for an urban development master plan that intends to create 36,000 new jobs, build 4,000 new homes, and provide £1.4bn economic uplift.  The plan identifies six 'Places for Growth' across the City, for the purposes of tourism, retail, business, learning, research, and a new creative sector.  

For the latter, the area/neighbourhood of Digbeth has been identified as the main area for the growing number of companies involved with digital technologies, design, TV production and arts.

Although development of the main infrastructure project has not yet started, a hospitality entrepreneur and an investor (the clients for this data science project) have requested a brief analysis of pre-existing popular venues and hotels around Digbeth and the likely site of the new railway station.  They are looking to strategically position 2-3 venues (1 boutique hotel*, and 1-2 restaurants) in the target area that would cater to both current popular demands and the future demands of the growing community of creative sector professionals living in or visiting the area.

(* A boutique hotel is typically a stylised hotel of 10-100 rooms with individualised 'unique selling points')

In particular, the clients (the target audience) have two questions at this pre-concept stage, and only a limited budget:

- What are the most popular (commonly visited) restaurant cuisines in and around the Digbeth area (i.e. the cluster of neighbourhoods) of Birmingham City, UK?
- What is the closest non-chain hotels near to (or in) Digbeth, Birmingham? How far from the Digbeth area is it? What is its customer rating?

## Data

This project will primarily utilise location and venue (visit and rating) data available through Foursquare.  Foursquare is a location technology company that, in 2009, developed a crowd-sourcing 'check-in' system that make sense of where phones go, and offers a propietary dataset (called Pilgrim) built upon over 13+ billion check-ins.

The project also uses a composite dataset that contains the postcode districts for Birmingham, their respective latitudes and longitudes, and estimated population by postcode (based upon 2011 UK census data).  This dataset was extracted from data curated by Chris Bell on www.dougal.co.uk (for which, the author is extremely grateful), and is available on a public domain licence. The data has been loaded directly into the IBM Watson Studio project, and connected with this notebook.

Lastly, the contextual data for the background of this project (i.e. information about HS2 and Birmingham City's master development plan), but not used in the data analysis of this project, has been taken from the Birmingham City Council website (www.birmingham.gov.uk/).

## Preliminaries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

import requests # library to handle requests
from bs4 import BeautifulSoup
import csv
import json
import xml

from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
#Connect with Foursquare API

CLIENT_ID = 'QNBM2LMRAUNUXJCPPBXC0WZQJIJQMBVURYLWBCXM1FQOEQPX' # your Foursquare ID
CLIENT_SECRET = 'MPACHLKKDKKTVJJPNMI32VEAC3CPM5WL2ZRF5Z2A2Z0313J2' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 1000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QNBM2LMRAUNUXJCPPBXC0WZQJIJQMBVURYLWBCXM1FQOEQPX
CLIENT_SECRET:MPACHLKKDKKTVJJPNMI32VEAC3CPM5WL2ZRF5Z2A2Z0313J2


## Loading Data

In [83]:
#Read in pre-loaded data from the IBM Watson Studio project

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

client_ce8198a121a849d89097425d2c3d8ebd = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='mcvvOvgDMxp7SabKMo-Tt1KzWhcVshi061CzH_ofLrwG',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_ce8198a121a849d89097425d2c3d8ebd.get_object(Bucket='boutiquehotelsinhs2erabirmingham-donotdelete-pr-ylwa9fq70kklgv',Key='B-postcodes.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

,Postcode,Latitude,Longitude,Town/Area,Region,Population,Households
0,B1,52.4796,-1.90778,"Birmingham City Centre, Ladywood",Birmingham,8514.0,4526.0
1,B2,52.4863,-1.89732,"Birmingham City Centre, Ladywood",Birmingham,655.0,473.0
2,B3,52.4823,-1.90288,"Birmingham City Centre, Ladywood",Birmingham,2226.0,1406.0
3,B4,52.4838,-1.89373,"Birmingham City Centre, Ladywood",Birmingham,4337.0,465.0
4,B5,52.4722,-1.89683,"Digbeth, Highgate, Lee Bank",Birmingham,12156.0,5139.0


## Mapping and Analysis

In [84]:
#Fix location and find coordinates for Birmingham, UK
address = 'Birmingham, UK'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Birmingham is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Birmingham is 52.4796992, -1.9026911.


In [85]:
# Focus only on Birmingham City, and not the wider county
df_data_1 = df_data_1[df_data_1['Region'] == 'Birmingham'].reset_index(drop=True)


In [86]:
# create map of Birmingham using latitude and longitude values
map_birmingham = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_data_1['Latitude'], df_data_1['Longitude'], df_data_1['Town/Area'], df_data_1['Region']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_birmingham)  
    
map_birmingham

In [87]:
#Formulate function to gather venue data, including name, location, and categories

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [88]:
#Create dataset for venues in Birmingham neighborhoods

birmingham_venues = getNearbyVenues(names=df_data_1['Town/Area'],
                                   latitudes=df_data_1['Latitude'],
                                   longitudes=df_data_1['Longitude']
                                  )

Digbeth, Highgate, Lee Bank
Aston
Nechells
Washwood Heath, Ward End, Saltley
Bordesley Green
Small Heath
Sparkhill, Tyseley
Balsall Heath, Sparkbrook, Highgate
Moseley, Billesley
Kings Heath, Yardley Wood, Brandwood, Druids Heath, Warstock
Edgbaston, Lee Bank
Edgbaston, Ladywood
Harborne
Winson Green, Hockley
Lozells, Newtown, Birchfield
Birchfield, Handsworth Wood Perry Barr
Handsworth
Kings Norton
Erdington, Short Heath
Erdington, Tyburn
Yardley
Sheldon, Yardley
Acocks Green
Hall Green
Selly Oak, Bournbrook, Selly Park, Weoley Castle, California
Bournville, Cotteridge, Stirchley
Northfield
Woodgate, Bartley Green, Quinton, California
Kitts Green, Stechford
Shard End, Buckland End
Castle Vale
Kings Norton
Perry Barr, Great Barr, Hamstead
Great Barr, Hamstead, Pheasey
Perry Barr, Kingstanding, Great Barr
Rednal, Rubery
Sutton Coldfield town centre, Maney, Wylde Green
Boldmere, New Oscott, Wylde Green
Four Oaks, Mere Green, Little Aston, Streetly
Sutton Trinity, Falcon Lodge, Rectory
Wa

In [89]:
#Check shape and check head of new dataframe

print(birmingham_venues.shape)
birmingham_venues.head()

(212, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Digbeth, Highgate, Lee Bank",52.4722,-1.89683,Quarter Horse Coffee,52.471454,-1.899532,Coffee Shop
1,"Digbeth, Highgate, Lee Bank",52.4722,-1.89683,Birmingham Hippodrome,52.474471,-1.897573,Theater
2,"Digbeth, Highgate, Lee Bank",52.4722,-1.89683,The Diskery,52.472253,-1.899471,Record Shop
3,"Digbeth, Highgate, Lee Bank",52.4722,-1.89683,Topokki (떡볶이),52.474059,-1.896732,Korean Restaurant
4,"Digbeth, Highgate, Lee Bank",52.4722,-1.89683,Eden,52.471721,-1.893867,Gay Bar


In [90]:
#Check number of venues by each neighborhood / postcode

birmingham_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Acocks Green,12,12,12,12,12,12
Aston,3,3,3,3,3,3
"Balsall Heath, Sparkbrook, Highgate",6,6,6,6,6,6
"Birchfield, Handsworth Wood Perry Barr",1,1,1,1,1,1
"Boldmere, New Oscott, Wylde Green",4,4,4,4,4,4
Bordesley Green,4,4,4,4,4,4
"Bournville, Cotteridge, Stirchley",6,6,6,6,6,6
Castle Vale,5,5,5,5,5,5
"Digbeth, Highgate, Lee Bank",38,38,38,38,38,38


In [91]:
print('There are {} uniques categories.'.format(len(birmingham_venues['Venue Category'].unique())))

There are 86 uniques categories.


In [92]:
# one hot encoding
birmingham_onehot = pd.get_dummies(birmingham_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
birmingham_onehot['Neighborhood'] = birmingham_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [birmingham_onehot.columns[-1]] + list(birmingham_onehot.columns[:-1])
birmingham_onehot = birmingham_onehot[fixed_columns]

birmingham_onehot.head()

,Neighborhood,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,Bar,Beer Bar,Beer Store,Bookstore,...,Sports Club,Stadium,Supermarket,Tapas Restaurant,Thai Restaurant,Theater,Train Station,Turkish Restaurant,Vietnamese Restaurant,Warehouse Store
0,"Digbeth, Highgate, Lee Bank",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Digbeth, Highgate, Lee Bank",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,"Digbeth, Highgate, Lee Bank",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Digbeth, Highgate, Lee Bank",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Digbeth, Highgate, Lee Bank",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
#Separate by venue category into one-hot coding, and check shape
birmingham_onehot.shape

(212, 87)

In [94]:
#Confirm sum totals of each category type
birmingham_onehot.sum()

Neighborhood                  Digbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee BankDigbeth, Highgate, Lee Ba

In [95]:
#Create dataframe on proportion of venue categories by neighborhood

birmingham_grouped = birmingham_onehot.groupby('Neighborhood').mean().reset_index()
birmingham_grouped

,Neighborhood,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,Bar,Beer Bar,Beer Store,Bookstore,...,Sports Club,Stadium,Supermarket,Tapas Restaurant,Thai Restaurant,Theater,Train Station,Turkish Restaurant,Vietnamese Restaurant,Warehouse Store
0,Acocks Green,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.083333,0.000000,0.000000,...,0.00,0.00,0.166667,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.083333
1,Aston,0.000000,0.000000,0.333333,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
2,"Balsall Heath, Sparkbrook, Highgate",0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
3,"Birchfield, Handsworth Wood Perry Barr",0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
4,"Boldmere, New Oscott, Wylde Green",0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
5,Bordesley Green,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.500000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
6,"Bournville, Cotteridge, Stirchley",0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.166667,0.166667,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000
7,Castle Vale,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.200000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.200000
8,"Digbeth, Highgate, Lee Bank",0.000000,0.000000,0.000000,0.0,0.026316,0.0,0.000000,0.000000,0.026316,...,0.00,0.00,0.000000,0.000000,0.026316,0.026316,0.00,0.0,0.026316,0.000000
9,"Edgbaston, Ladywood",0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000


In [96]:
#Check shape of new dataframe

birmingham_grouped.shape

(38, 87)

In [97]:
#Analyse frequency of venues by neighborhood

num_top_venues = 10

for hood in birmingham_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = birmingham_grouped[birmingham_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Acocks Green----
                    venue  freq
0  Pub                     0.17
1  Supermarket             0.17
2  Hotel                   0.08
3  Grocery Store           0.08
4  Furniture / Home Store  0.08
5  Coffee Shop             0.08
6  Sandwich Place          0.08
7  Bowling Alley           0.08
8  Warehouse Store         0.08
9  Beer Bar                0.08


----Aston----
                   venue  freq
0  Music Venue            0.33
1  Auto Garage            0.33
2  Park                   0.33
3  Optical Shop           0.00
4  Plaza                  0.00
5  Pizza Place            0.00
6  Photography Studio     0.00
7  Pet Store              0.00
8  Persian Restaurant     0.00
9  Performing Arts Venue  0.00


----Balsall Heath, Sparkbrook, Highgate----
                   venue  freq
0  Fast Food Restaurant   0.33
1  Performing Arts Venue  0.17
2  Café                   0.17
3  Grocery Store          0.17
4  Electronics Store      0.17
5  Asian Restaurant       0.00
6  Park

In [98]:
#Formulate function to return most comon venues in a given neighborhood

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [99]:
#Create a sorted dataframe of top venues by postcode/neighborhood in Birmingham

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = birmingham_grouped['Neighborhood']

for ind in np.arange(birmingham_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(birmingham_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acocks Green,Pub,Supermarket,Warehouse Store,Bowling Alley,Grocery Store,Furniture / Home Store,Hotel,Sandwich Place,Coffee Shop,Beer Bar
1,Aston,Auto Garage,Park,Music Venue,Warehouse Store,Gastropub,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service,Furniture / Home Store
2,"Balsall Heath, Sparkbrook, Highgate",Fast Food Restaurant,Electronics Store,Performing Arts Venue,Grocery Store,Café,Warehouse Store,Gas Station,Farmers Market,Fish & Chips Shop,Food Service
3,"Birchfield, Handsworth Wood Perry Barr",Grocery Store,Warehouse Store,Gay Bar,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service,Furniture / Home Store,Gas Station
4,"Boldmere, New Oscott, Wylde Green",Gym Pool,Coffee Shop,Park,Warehouse Store,Gas Station,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service


In [100]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acocks Green,Pub,Supermarket,Warehouse Store,Bowling Alley,Grocery Store,Furniture / Home Store,Hotel,Sandwich Place,Coffee Shop,Beer Bar
1,Aston,Auto Garage,Park,Music Venue,Warehouse Store,Gastropub,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service,Furniture / Home Store
2,"Balsall Heath, Sparkbrook, Highgate",Fast Food Restaurant,Electronics Store,Performing Arts Venue,Grocery Store,Café,Warehouse Store,Gas Station,Farmers Market,Fish & Chips Shop,Food Service
3,"Birchfield, Handsworth Wood Perry Barr",Grocery Store,Warehouse Store,Gay Bar,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service,Furniture / Home Store,Gas Station
4,"Boldmere, New Oscott, Wylde Green",Gym Pool,Coffee Shop,Park,Warehouse Store,Gas Station,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service
5,Bordesley Green,Supermarket,Bus Stop,Furniture / Home Store,Warehouse Store,Gastropub,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service
6,"Bournville, Cotteridge, Stirchley",Indian Restaurant,Grocery Store,Park,Beer Store,Beer Bar,Go Kart Track,Gym,Gym / Fitness Center,Gym Pool,Greek Restaurant
7,Castle Vale,Warehouse Store,Electronics Store,Gym Pool,Chinese Restaurant,Supermarket,Gastropub,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service
8,"Digbeth, Highgate, Lee Bank",Gay Bar,Chinese Restaurant,Hotel,Music Venue,Korean Restaurant,Japanese Restaurant,Burger Joint,Farmers Market,Latin American Restaurant,Coffee Shop
9,"Edgbaston, Ladywood",Convenience Store,Sandwich Place,Grocery Store,Insurance Office,Middle Eastern Restaurant,Music Store,Fast Food Restaurant,Electronics Store,Persian Restaurant,Chinese Restaurant


In [101]:
# set number of clusters
kclusters = 5

birmingham_grouped_clustering = birmingham_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(birmingham_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [102]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [103]:
birmingham_merged = df_data_1

birmingham_merged = birmingham_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Town/Area')

birmingham_merged.head() 

,Postcode,Latitude,Longitude,Town/Area,Region,Population,Households,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,B5,52.4722,-1.89683,"Digbeth, Highgate, Lee Bank",Birmingham,12156.0,5139.0,0.0,Gay Bar,Chinese Restaurant,Hotel,Music Venue,Korean Restaurant,Japanese Restaurant,Burger Joint,Farmers Market,Latin American Restaurant,Coffee Shop
1,B6,52.5025,-1.88686,Aston,Birmingham,19507.0,5886.0,0.0,Auto Garage,Park,Music Venue,Warehouse Store,Gastropub,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service,Furniture / Home Store
2,B7,52.4938,-1.87437,Nechells,Birmingham,8554.0,3444.0,0.0,Convenience Store,Auto Garage,Gym / Fitness Center,Grocery Store,Gastropub,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service
3,B8,52.4900,-1.84277,"Washwood Heath, Ward End, Saltley",Birmingham,42278.0,11987.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,B9,52.4781,-1.85285,Bordesley Green,Birmingham,24915.0,7548.0,0.0,Supermarket,Bus Stop,Furniture / Home Store,Warehouse Store,Gastropub,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service


In [104]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [105]:
birmingham_merged.loc[birmingham_merged['Cluster Labels'] == 0, birmingham_merged.columns[[1] + list(range(5, birmingham_merged.shape[1]))]]


,Latitude,Population,Households,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,52.4722,12156.0,5139.0,0.0,Gay Bar,Chinese Restaurant,Hotel,Music Venue,Korean Restaurant,Japanese Restaurant,Burger Joint,Farmers Market,Latin American Restaurant,Coffee Shop
1,52.5025,19507.0,5886.0,0.0,Auto Garage,Park,Music Venue,Warehouse Store,Gastropub,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service,Furniture / Home Store
2,52.4938,8554.0,3444.0,0.0,Convenience Store,Auto Garage,Gym / Fitness Center,Grocery Store,Gastropub,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service
4,52.4781,24915.0,7548.0,0.0,Supermarket,Bus Stop,Furniture / Home Store,Warehouse Store,Gastropub,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service
5,52.4701,26004.0,7108.0,0.0,Café,Sandwich Place,Indian Restaurant,Mosque,Fast Food Restaurant,Middle Eastern Restaurant,Turkish Restaurant,Gym / Fitness Center,Grocery Store,Furniture / Home Store
6,52.4554,44391.0,12602.0,0.0,Italian Restaurant,Pakistani Restaurant,Gas Station,Warehouse Store,Gastropub,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service
7,52.4609,20548.0,6649.0,0.0,Fast Food Restaurant,Electronics Store,Performing Arts Venue,Grocery Store,Café,Warehouse Store,Gas Station,Farmers Market,Fish & Chips Shop,Food Service
8,52.4351,37672.0,15392.0,0.0,Auto Workshop,Golf Course,Warehouse Store,Gay Bar,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service,Furniture / Home Store
9,52.4188,43099.0,18063.0,0.0,Photography Studio,Gym / Fitness Center,Grocery Store,Chinese Restaurant,Restaurant,Cricket Ground,Furniture / Home Store,Deli / Bodega,Electronics Store,Farmers Market
10,52.4679,17872.0,6220.0,0.0,Coffee Shop,Asian Restaurant,Botanical Garden,Hotel,Pub,Café,Restaurant,Gastropub,Supermarket,Tapas Restaurant


In [106]:
birmingham_merged.loc[birmingham_merged['Cluster Labels'] == 1, birmingham_merged.columns[[1] + list(range(5, birmingham_merged.shape[1]))]]


,Latitude,Population,Households,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,52.3885,30273.0,12810.0,1.0,Park,Warehouse Store,Gastropub,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service,Furniture / Home Store,Gas Station


In [107]:
birmingham_merged.loc[birmingham_merged['Cluster Labels'] == 2, birmingham_merged.columns[[1] + list(range(5, birmingham_merged.shape[1]))]]


,Latitude,Population,Households,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,52.4957,18941.0,8112.0,2.0,IT Services,Gay Bar,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service,Furniture / Home Store,Gas Station,Gastropub


In [108]:
birmingham_merged.loc[birmingham_merged['Cluster Labels'] == 3, birmingham_merged.columns[[1] + list(range(5, birmingham_merged.shape[1]))]]


,Latitude,Population,Households,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,52.4509,38357.0,16209.0,3.0,Fast Food Restaurant,Warehouse Store,Gay Bar,Electronics Store,Farmers Market,Fish & Chips Shop,Food Service,Furniture / Home Store,Gas Station,Gastropub


In [109]:
birmingham_merged.loc[birmingham_merged['Cluster Labels'] == 4, birmingham_merged.columns[[1] + list(range(5, birmingham_merged.shape[1]))]]


,Latitude,Population,Households,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,52.4915,16739.0,6389.0,4.0,Bar,Warehouse Store,Gay Bar,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Service,Furniture / Home Store,Gas Station


In [110]:
birmingham_merged.loc[birmingham_merged['Cluster Labels'] == 5, birmingham_merged.columns[[1] + list(range(5, birmingham_merged.shape[1]))]]


,Latitude,Population,Households,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [111]:
address = 'Digbeth, Birmingham, UK'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

52.4734488 -1.8871192


In [112]:
search_query = 'Hotel'
radius = 5000
print(search_query + ' .... OK!')

Hotel .... OK!


In [113]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=QNBM2LMRAUNUXJCPPBXC0WZQJIJQMBVURYLWBCXM1FQOEQPX&client_secret=MPACHLKKDKKTVJJPNMI32VEAC3CPM5WL2ZRF5Z2A2Z0313J2&ll=52.4734488,-1.8871192&v=20180604&query=Hotel&radius=5000&limit=1000'

In [114]:
results = requests.get(url).json()

In [115]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'Hotel', 'pluralName': 'Hotels', 'shortName': 'Hotel', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_', 'suffix': '.png'}, 'primary': True}]",False,570673e7498e909686bcccce,"160 Wharfside Street, The Mailbox",GB,Birmingham,United Kingdom,NaN,1302,"[160 Wharfside Street, The Mailbox, Birmingham, West Midlands, B1 1RL, United Kingdom]","[{'label': 'display', 'lat': 52.475198, 'lng': -1.906108}]",52.475198,-1.906108,NaN,B1 1RL,West Midlands,AC Hotel by Marriott Birmingham,v-1583454846,422703144
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'Hotel', 'pluralName': 'Hotels', 'shortName': 'Hotel', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_', 'suffix': '.png'}, 'primary': True}]",False,4b058829f964a52077b522e3,"12 Holloway Circus, Queensway",GB,Birmingham,United Kingdom,NaN,922,"[12 Holloway Circus, Queensway, Birmingham, West Midlands, B1 1BT, United Kingdom]","[{'label': 'display', 'lat': 52.47547677455434, 'lng': -1.9003098412304098}]",52.475477,-1.900310,NaN,B1 1BT,West Midlands,"Radisson Blu Hotel, Birmingham",v-1583454846,NaN
2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'Hotel', 'pluralName': 'Hotels', 'shortName': 'Hotel', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_', 'suffix': '.png'}, 'primary': True}]",False,4b058828f964a5205db522e3,New St,GB,Birmingham,United Kingdom,NaN,914,"[New St, Birmingham, West Midlands, B2 4RX, United Kingdom]","[{'label': 'display', 'lat': 52.47899629983553, 'lng': -1.8970683618255177}]",52.478996,-1.897068,NaN,B2 4RX,West Midlands,Britannia Hotel,v-1583454846,NaN
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'Hotel', 'pluralName': 'Hotels', 'shortName': 'Hotel', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_', 'suffix': '.png'}, 'primary': True}]",False,4ba93479f964a5201a143ae3,21 Ladywell Walk,GB,Birmingham,United Kingdom,NaN,655,"[21 Ladywell Walk, Birmingham, West Midlands, B5 4ST, United Kingdom]","[{'label': 'display', 'lat': 52.47504220011472, 'lng': -1.8964303842640056}]",52.475042,-1.896430,NaN,B5 4ST,West Midlands,Ibis Hotel,v-1583454846,NaN
4,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'Hotel', 'pluralName': 'Hotels', 'shortName': 'Hotel', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_', 'suffix': '.png'}, 'primary': True}]",False,501d14ade4b08947b51d0f04,1 Great Colmore Street,GB,Birmingham,United Kingdom,NaN,924,"[1 Great Colmore Street, Birmingham, West Midlands, B15 2AP, United Kingdom]","[{'label': 'display', 'lat': 52.471385481668946, 'lng': -1.9003275464563103}]",52.471385,-1.900328,NaN,B15 2AP,West Midlands,Ibis Budget Hotel,v-1583454846,49179518


In [116]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,AC Hotel by Marriott Birmingham,Hotel,"160 Wharfside Street, The Mailbox",GB,Birmingham,United Kingdom,NaN,1302,"[160 Wharfside Street, The Mailbox, Birmingham, West Midlands, B1 1RL, United Kingdom]","[{'label': 'display', 'lat': 52.475198, 'lng': -1.906108}]",52.475198,-1.906108,NaN,B1 1RL,West Midlands,570673e7498e909686bcccce
1,"Radisson Blu Hotel, Birmingham",Hotel,"12 Holloway Circus, Queensway",GB,Birmingham,United Kingdom,NaN,922,"[12 Holloway Circus, Queensway, Birmingham, West Midlands, B1 1BT, United Kingdom]","[{'label': 'display', 'lat': 52.47547677455434, 'lng': -1.9003098412304098}]",52.475477,-1.900310,NaN,B1 1BT,West Midlands,4b058829f964a52077b522e3
2,Britannia Hotel,Hotel,New St,GB,Birmingham,United Kingdom,NaN,914,"[New St, Birmingham, West Midlands, B2 4RX, United Kingdom]","[{'label': 'display', 'lat': 52.47899629983553, 'lng': -1.8970683618255177}]",52.478996,-1.897068,NaN,B2 4RX,West Midlands,4b058828f964a5205db522e3
3,Ibis Hotel,Hotel,21 Ladywell Walk,GB,Birmingham,United Kingdom,NaN,655,"[21 Ladywell Walk, Birmingham, West Midlands, B5 4ST, United Kingdom]","[{'label': 'display', 'lat': 52.47504220011472, 'lng': -1.8964303842640056}]",52.475042,-1.896430,NaN,B5 4ST,West Midlands,4ba93479f964a5201a143ae3
4,Ibis Budget Hotel,Hotel,1 Great Colmore Street,GB,Birmingham,United Kingdom,NaN,924,"[1 Great Colmore Street, Birmingham, West Midlands, B15 2AP, United Kingdom]","[{'label': 'display', 'lat': 52.471385481668946, 'lng': -1.9003275464563103}]",52.471385,-1.900328,NaN,B15 2AP,West Midlands,501d14ade4b08947b51d0f04
5,Ibis Hotel,Hotel,"1 Bordesley Park Road, Bordesley",GB,Birmingham,United Kingdom,NaN,964,"[1 Bordesley Park Road, Bordesley, Birmingham, West Midlands, B10 0PD, United Kingdom]","[{'label': 'display', 'lat': 52.47243788179971, 'lng': -1.872997934188561}]",52.472438,-1.872998,NaN,B10 0PD,West Midlands,4c6874c331ba2d7f86eaf772
6,Macdonald Burlington Hotel,Hotel,126 New St,GB,Birmingham,United Kingdom,btw Lower Temple St & Stephensons St,1011,"[126 New St (btw Lower Temple St & Stephensons St), Birmingham, West Midlands, B2 4JQ, United Kingdom]","[{'label': 'display', 'lat': 52.47904101927383, 'lng': -1.8988682803135941}]",52.479041,-1.898868,NaN,B2 4JQ,West Midlands,4b058828f964a5203ab522e3
7,Copthorne Hotel Birmingham,Hotel,"Paradise Circus, Paradise Place",GB,Warwickshire,United Kingdom,Centenary Square,1518,"[Paradise Circus, Paradise Place (Centenary Square), Warwickshire, B3 3HJ, United Kingdom]","[{'label': 'display', 'lat': 52.48033191682519, 'lng': -1.906459115451753}]",52.480332,-1.906459,NaN,B3 3HJ,Warwickshire,51818544c84c71399407eca0
8,Hotel Indigo Birmingham,Hotel,"The Cube, Wharfside St",GB,Birmingham,United Kingdom,at Commercial St,1329,"[The Cube, Wharfside St (at Commercial St), Birmingham, West Midlands, B1 1RS, United Kingdom]","[{'label': 'display', 'lat': 52.4749228, 'lng': -1.9065749}]",52.474923,-1.906575,NaN,B1 1RS,West Midlands,4eee6a37003937534527c5ba
9,Hotel Chocolat,Café,117 New St,GB,Birmingham,United Kingdom,NaN,1037,"[117 New St, Birmingham, West Midlands, B2 4JH, United Kingdom]","[{'label': 'display', 'lat': 52.479125, 'lng': -1.899262}]",52.479125,-1.899262,NaN,B2 4JH,West Midlands,5a05d9d10457b70895825389


In [117]:
dataframe_filtered.sort_values('distance', inplace=True)

In [118]:
dataframe_filtered.name

15    Hotel car Park                                                      
19    Paragon Hotel                                                       
17    Hotel Du Bullring                                                   
28    Birmingham Central Hotel                                            
3     Ibis Hotel                                                          
22    Hotel Chocolat                                                      
27    The Grand Hotel Birmingham                                          
30    The Fountain Hotel                                                  
31    The Bristol Hotel                                                   
16    Clayton Hotel                                                       
32    F1 Hotel                                                            
2     Britannia Hotel                                                     
25    Radisson Blu Hotel Bar.                                             
1     Radisson Blu Hotel,

In [119]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [130]:
venue_id = '4eee6a37003937534527c5ba' # ID of Indigo Hotel
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4eee6a37003937534527c5ba?client_id=QNBM2LMRAUNUXJCPPBXC0WZQJIJQMBVURYLWBCXM1FQOEQPX&client_secret=MPACHLKKDKKTVJJPNMI32VEAC3CPM5WL2ZRF5Z2A2Z0313J2&v=20180604'

In [131]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'storeId', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes', 'bestPhoto', 'colors'])


{'id': '4eee6a37003937534527c5ba',
 'name': 'Hotel Indigo Birmingham',
 'contact': {'phone': '+441216432010',
  'formattedPhone': '+44 121 643 2010',
  'twitter': 'hotelindigo',
  'facebook': '105078596190067',
  'facebookUsername': 'hotelindigo',
  'facebookName': 'Hotel Indigo'},
 'location': {'address': 'The Cube, Wharfside St',
  'crossStreet': 'at Commercial St',
  'lat': 52.4749228,
  'lng': -1.9065749,
  'labeledLatLngs': [{'label': 'display',
    'lat': 52.4749228,
    'lng': -1.9065749}],
  'postalCode': 'B1 1RS',
  'cc': 'GB',
  'city': 'Birmingham',
  'state': 'West Midlands',
  'country': 'United Kingdom',
  'formattedAddress': ['The Cube, Wharfside St (at Commercial St)',
   'Birmingham',
   'West Midlands',
   'B1 1RS',
   'United Kingdom']},
 'canonicalUrl': 'https://foursquare.com/v/hotel-indigo-birmingham/4eee6a37003937534527c5ba',
 'categories': [{'id': '4bf58dd8d48988d1fa931735',
   'name': 'Hotel',
   'pluralName': 'Hotels',
   'shortName': 'Hotel',
   'icon': {'pre

In [132]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.9


In [133]:
result['response']['venue']['tips']['count']

9

In [134]:
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e619f625428900021980235'},
 'response': {'tips': {'count': 9,
   'items': [{'id': '59bbacdb4b78c509238e630c',
     'createdAt': 1505471707,
     'text': "Gorgeous and modern hotel w the best views, bar on the top floor, and the nicest fitness center I've ever seen at a hotel. Highly recommend.",
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/59bbacdb4b78c509238e630c',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 0,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '85407986',
      'firstName': 'Kristen',
      'lastName': 'V',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/85407986-IN0TU04JPYNFW5U3.jpg'}},
     'authorInteractionType': 'liked'}]}}}

In [135]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [136]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"Gorgeous and modern hotel w the best views, bar on the top floor, and the nicest fitness center I've ever seen at a hotel. Highly recommend.",0,0,59bbacdb4b78c509238e630c,Kristen,V,NaN,85407986


In [129]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

4.7
